In [ ]:
%load_ext autoreload
%autoreload 2

# export libraries
from pathlib import Path
import pandas as pd
import sys
import torch
import warnings

# suppress warnings in notebook
warnings.filterwarnings('ignore')

# add src to path
sys.path.append(str(Path('../').resolve()))

from src.models.llm_handler import LLMFactory
from src.models.extraction.llm_extractor import LLMExtractor
from src.models.classification.intent_classifier import LLMIntentClassifier

print('CUDA Available:', torch.cuda.is_available())
print('CUDA Version:', torch.version.cuda)
print('GPU:', torch.cuda.get_device_name(0))

In [2]:
df = pd.read_csv("../data/processed/cleaned_messages.csv")

print("Dataframe shape:", df.shape)
df.head(10)

Dataframe shape: (18949, 3)


,sender,message,timestamp
0,"dr. Andreas C.N., Fp.B.",Siap terimakasih sudah diadd di grup 🙏🏻,2020-08-02 11:52:41
1,Lenny Pandjidharma,sami2 ...,2020-08-02 11:52:51
2,Lenny Pandjidharma,untuk peraturan group dan tata pelaksanaan ......,2020-08-02 11:53:17
3,Oma Lisa,Thanks Lenny.t Lisa ikut ya spytdk cpt pikun a...,2020-08-02 12:20:41
4,Mfitri,"Ok,makasih ci 🙏",2020-08-02 12:22:35
5,Sim Ay Tjan,Thanks Len.🙏🏼,2020-08-02 12:51:17
6,Tjunfebelyana,Thanks Len🙏,2020-08-02 13:01:32
7,Oma Lisa,Mulak kapan dan jam brp Lenny.Gbu txs,2020-08-02 14:22:31
8,Lenny Pandjidharma,"Dimulainya besok, Tante Lisa.",2020-08-02 14:26:58
9,Oma Lisa,Ok,2020-08-02 14:37:45


In [3]:
messages = df["message"].astype(str).tolist()

In [4]:
handler = LLMFactory.create_handler('komodo')

Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.32s/it]
Device set to use cuda:0


In [5]:
# Prompts

with open("..\prompts\extract_progress_prompt.txt", "r", encoding="utf-8") as f:
    extract_command = f.read()

with open("..\prompts\intent_classification_prompt.txt", "r", encoding="utf-8") as f:
    intent_command = f.read()

In [6]:
test_message = "Kej  50- Ul 2 done"

result = handler.generate(
    prompt=test_message,
    system_message=extract_command,
    mode="extraction"
)

result

Processing batches: 100%|██████████| 1/1 [00:11<00:00, 11.98s/msg]


'[{"book_text":"Kej","start_chapter":50,"end_chapter":50,"raw_text":"Kej 50","confidence":1.0,"source":"llm"},{"book_text":"Ul","start_chapter":2,"end_chapter":2,"raw_text":"Ul 2","confidence":1.0,"source":"llm"}]'

In [6]:
# Test on small sample first
test_messages = messages[30:50]

test_outputs = handler.generate_batch(
    prompts=test_messages,
    system_message=intent_command,
    mode='intent'
)

# Verify quality
for msg, out in zip(test_messages, test_outputs):
    print(f"Input: {msg}")
    print(f"Output: {out}\n")

Processing batches: 100%|██████████| 20/20 [00:41<00:00,  2.08s/msg]

Input: Kej 1-2 done
Output: {"is_progress_report":true,"confidence":1.0}

Input: Kej 1-2 done
Output: {"is_progress_report":true,"confidence":1.0}

Input: Kej 1-2 done
Output: {"is_progress_report":true,"confidence":1.0}

Input: Kej 1-2 done
Output: {"is_progress_report":true,"confidence":1.0}

Input: Kej 1- 2 selesai.🙏
Output: {"is_progress_report":true,"confidence":0.9}

Input: Kej 1-2 done
Output: {"is_progress_report":true,"confidence":1.0}

Input: Kej 1-2 done
Output: {"is_progress_report":true,"confidence":1.0}

Input: Kej 1-2 selesai
Output: {"is_progress_report":true,"confidence":1.0}

Input: Kej1-2 done
Output: {"is_progress_report":true,"confidence":1.0}

Input: Kej 1-2 done
Output: {"is_progress_report":true,"confidence":1.0}

Input: _Kej 1-2_ ✓
Output: {"is_progress_report":true,"confidence":1.0}

Input: Kej 1-2✓
Output: {"is_progress_report":true,"confidence":1.0}

Input: Kej 1-2 done
Output: {"is_progress_report":true,"confidence":1.0}

Input: https://youtu.be/6t3IrHfO3nc

In [7]:
test_outputs_2 = handler.generate_batch(
    prompts=test_messages,
    system_message=extract_command,
    mode='extraction'
)

# Verify quality
for msg, out in zip(test_messages, test_outputs_2):
    print(f"Input: {msg}")
    print(f"Output: {out}\n")

Processing batches: 100%|██████████| 20/20 [01:09<00:00,  3.49s/msg]

Input: Kej 1-2 done
Output: [{"book_text":"Kej","start_chapter":1,"end_chapter":2,"raw_text":"Kej 1-2","confidence":1.0,"source":"llm"}]

Input: Kej 1-2 done
Output: [{"book_text":"Kej","start_chapter":1,"end_chapter":2,"raw_text":"Kej 1-2","confidence":1.0,"source":"llm"}]

Input: Kej 1-2 done
Output: [{"book_text":"Kej","start_chapter":1,"end_chapter":2,"raw_text":"Kej 1-2","confidence":1.0,"source":"llm"}]

Input: Kej 1-2 done
Output: [{"book_text":"Kej","start_chapter":1,"end_chapter":2,"raw_text":"Kej 1-2","confidence":1.0,"source":"llm"}]

Input: Kej 1- 2 selesai.🙏
Output: [{"book_text":"Kej","start_chapter":1,"end_chapter":2,"raw_text":"Kej 1- 2 selesai.🙏","confidence":1.0,"source":"llm"}]

Input: Kej 1-2 done
Output: [{"book_text":"Kej","start_chapter":1,"end_chapter":2,"raw_text":"Kej 1-2","confidence":1.0,"source":"llm"}]

Input: Kej 1-2 done
Output: [{"book_text":"Kej","start_chapter":1,"end_chapter":2,"raw_text":"Kej 1-2","confidence":1.0,"source":"llm"}]

Input: Kej 1-2 se

In [8]:
# test cases
test_messages = [

    "Kej 1-2 done",
    "Kel 3-4 selesai",
    "Im 5-6 ✓",
    "Bil 10-12 done",
    "Ul 6-8",
    "Yos 1-4 selesai",
    "Hak 3-4 done",
    "Rut 1-4 ✓",
    "1Sam 6-10 selesai",
    "2Sam 22-24 done",
    
    "Kejadian 1-3 done",
    "Keluaran 12-14 selesai",
    "Imamat 16-18 done",
    "Bilangan 20-22 ✓",
    "Ulangan 28-30",
    
    "Kej 1-2 🙏",
    "Mat 5-7 ✅",
    "Yoh 3-4 done 🔥",
    "Roma 7-8 selesai 💯",
    "Wahyu 21-22 ✓✓",
    
    # Single reference with context
    "Hari ini baca Kejadian 1-2 selesai",
    "Sudah selesai baca Keluaran 12-15 pagi ini",
    "Bacaan hari ini Mazmur 21-23 sudah selesai",
    "Pagi ini berhasil baca Amsal 3-5 done",
    "Malam ini baca Yesaya 51-53 selesai semua",
    
    # Multiple references
    "Kej 1-2 done, Kej 3-4 done",
    "Mat 5-7 selesai, Mat 8-10 selesai",
    "Yoh 1-2 done, Yoh 3-4 done, Yoh 5-6 done",
    "Roma 8-10 selesai, roma 11-12 juga done",
    "1 Korintus 13-14 done, 1 korintus 15-16 done",
    
    # With markdown formatting
    "_Kej 1-2_ selesai dibaca hari ini",
    "*Mazmur 118-119* done sudah selesai",
    "**Yohanes 3:16** pasal 3 sudah selesai",
    "~~belum~~ sudah baca Kejadian 1-5 done",
    
    # Casual language
    "Kej 1-2 udah selesai gais",
    "Bacaan Matius 5-7 udh done nih",
    "Yoh 3-4 kelar dibaca tadi pagi",
    "Roma 8-10 finish udah saya baca",
    
    # Long progress reports
    "Hari ini sudah selesai membaca Kejadian pasal 1 sampai pasal 5 dengan penuh hikmat dan berkat",
    "Pagi yang diberkati, saya sudah menyelesaikan bacaan Alkitab hari ini yaitu Keluaran 12-15 dengan penuh syukur",
    "Puji Tuhan hari ini saya berhasil menyelesaikan bacaan Mazmur 1-10 semuanya lancar dan penuh berkat dari Tuhan",
    "Terima kasih Tuhan atas firman-Mu hari ini, sudah selesai baca Amsal 1-5 dan sangat memberkati hidup saya",
    "Hari ini bacaan yang luar biasa memberkati dari Yesaya pasal 40-45 sudah selesai saya baca dengan penuh perhatian",
    
    # Multiple references with details
    "Hari ini produktif banget, sudah selesai Kejadian 1-3 pagi, Kejadian 4-6 siang, dan Kejadian 7-9 malam done semua",
    "Bacaan hari Minggu: Matius 5-7 (khotbah di bukit) selesai pagi ini, Matius 8-10 (mujizat-mujizat) selesai sore done",
    "Renungan hari ini dari Yohanes 1-2 selesai, Yohanes 3-4 selesai, Yohanes 4-6 selesai semua lancar",
    "Puji Tuhan sudah menyelesaikan Roma 8-10 tentang roh dan Roma 11-12 tentang persembahan yang hidup done semua hari ini",
    "Puji Tuhan bacaan 1 Korintus 13-14 tentang kasih dan 1 Korintus 14-15 tentang kebangkitan sudah selesai dibaca dengan khusyuk",
    
    #short non-progress (Questions/Greetings)
    "Halo apa kabar?",
    "Selamat pagi",
    "Terima kasih ya",
    "Amin",
    "Syalom",
    "Baik pak",
    "Siap bu",
    "Ok terima kasih",
    "Saya setuju",
    "Mantap sekali",
    
    # medium non-progress (Questions about Bible)
    "Apa arti dari Yohanes 3:16?",
    "Tolong jelaskan maksud dari Kejadian pasal 1",
    "Siapa yang menulis kitab Roma?",
    "Kapan Paulus menulis surat Korintus?",
    "Apa makna dari Mazmur 23?",
    "Bagaimana cara memahami kitab Wahyu?",
    "Kenapa Yesus berkata seperti itu di Matius 5?",
    "Apa perbedaan Perjanjian Lama dan Baru?",
    "Siapa saja murid Yesus yang 12 orang itu?",
    "Dimana Tuhan Yesus dilahirkan?",
    
    # Personal questions
    "Hari ini saya belum sempat baca Alkitab",
    "Kapan kita bisa mulai rencana baca?",
    "Saya ingin belajar lebih dalam",
    "Bisakah kamu bantu saya?",
    "Apa rencana baca yang bagus untuk pemula?",
    
    # Long non-progress (Detailed questions/chat)
    "Saya ingin bertanya tentang makna dari kitab Kejadian pasal 1, apakah itu harus dipahami secara literal atau metaforis?",
    "Bisa tolong jelaskan konteks sejarah dari kitab Roma dan mengapa Paulus menulisnya untuk jemaat di Roma saat itu?",
    "Saya sedang bingung memahami kitab Wahyu, apakah ada cara yang lebih mudah untuk memahami simbol-simbol yang ada di dalamnya?",
    "Hari ini saya merasa down dan butuh penghiburan, apakah ada ayat-ayat Alkitab yang bisa menguatkan hati saya?",
    "Saya ingin membuat jadwal baca Alkitab selama 30 hari untuk pemula, kira-kira mulai dari kitab apa yang paling cocok?",
    
    # General chat
    "Terima kasih banyak atas bantuan dan dukungannya selama ini, semoga Tuhan memberkati pelayanan ini terus menerus",
    "Selamat pagi, semoga hari ini menjadi hari yang penuh berkat dan damai sejahtera dari Tuhan Yang Maha Esa",
    "Saya sangat senang bisa belajar Alkitab melalui aplikasi ini, mudah-mudahan semakin banyak fitur yang berguna",
    "Apakah ada komunitas atau grup diskusi Alkitab yang bisa saya ikuti untuk belajar bersama-sama dengan orang lain?",
    "Saya ingin berterima kasih kepada tim developer yang sudah membuat aplikasi ini dengan sangat baik dan bermanfaat",
    
    # Edge cases - mentions Bible but not progress
    "Saya suka sekali membaca Alkitab setiap hari",
    "Kejadian itu adalah kitab pertama dalam Alkitab",
    "Paulus menulis surat Roma dari penjara",
    "Yesus mengajar di bukit seperti di Matius 5",
    "Alkitab terdiri dari Perjanjian Lama dan Baru",
    
    # Tricky cases (Edge cases to test robustness)

    # Has Bible reference but not progress
    "Apakah Yohanes 3:16 itu ayat paling terkenal?",
    "Saya tertarik belajar tentang Kejadian 1-11",
    "Tolong carikan ayat tentang kasih di 1 Korintus 13",
    "Kemarin di gereja khotbahnya dari Matius 5-7",
    "Renungan saya hari ini tentang Roma 8",
    
    # Ambiguous - could be interpreted either way
    "Kejadian 1-2 sangat menarik untuk dipelajari",
    "Matius 5-7 adalah bagian favorit saya",
    "Yohanes 3 itu powerful banget",
    "Roma 8 itu chapter terbaik",
    "Wahyu 21-22 begitu indah",
    
    # Progress with extra context (should still be detected)
    "Btw Kej 1-2 done ya hari ini",
    "Oh iya, Mat 5-7 selesai sudah tadi pagi",
    "Update: Yoh 1-3 done semua nih",
    "FYI sudah selesai baca Roma 8-12",
    "Just info, Wahyu 19-22 selesai semua",
]

# Expected labels (for testing)
expected_labels = {
    # Progress reports (indices)
    "progress": [
        0, 1, 2, 3, 4, 5, 6, 7, 8, 9,  # Short abbreviated
        10, 11, 12, 13, 14,  # Short full name
        15, 16, 17, 18, 19,  # With emojis
        20, 21, 22, 23, 24,  # Medium with context
        25, 26, 27, 28, 29,  # Multiple references
        30, 31, 32, 33,  # Markdown
        34, 35, 36, 37,  # Casual
        38, 39, 40, 41, 42,  # Long detailed
        43, 44, 45, 46, 47,  # Long multiple
        # Last 5 ambiguous cases with "done/selesai" are progress
        95, 96, 97, 98, 99
    ],
    # Non-progress (all others)
}

print(f"Total messages: {len(test_messages)}")
print(f"Expected progress reports: {len(expected_labels['progress'])}")
print(f"Expected non-progress: {len(test_messages) - len(expected_labels['progress'])}")

Total messages: 103
Expected progress reports: 53
Expected non-progress: 50


In [9]:
extractor = LLMExtractor(handler, extract_command)
classifier = LLMIntentClassifier(handler, intent_command)

In [13]:
# Classify all messages
intent_outputs = classifier.classify_batch(messages=test_messages)

# Parse and filter progress reports
progress_messages = []
progress_indices = []

for i, intent_output in enumerate(intent_outputs):
    
    # Check if it's a progress report
    if intent_output.get("is_progress_report", False):
        progress_messages.append(test_messages[i])
        progress_indices.append(i)
    

print(f"Found {len(progress_messages)} progress reports out of {len(test_messages)} messages")

# extract only the progress reports
if progress_messages:
    extraction_outputs = extractor.extract_batch(messages=progress_messages)
    
    # Map back to original indices
    results = []
    for i, extract_output in enumerate(extraction_outputs):
        original_index = progress_indices[i]
        results.append({
            "index": original_index,
            "message": progress_messages[i],
            "intent": intent_outputs[original_index],
            "extraction": extract_output
        })
    
    print(f"Extracted references from {len(results)} messages")
else:
    print("No progress reports found, skipping extraction")
    results = []

Processing batches: 100%|██████████| 103/103 [02:19<00:00,  1.35s/msg]


Found 57 progress reports out of 103 messages


Processing batches: 100%|██████████| 57/57 [04:24<00:00,  4.64s/msg]

Extracted references from 57 messages


In [14]:
intent_outputs

[{'is_progress_report': True, 'confidence': 1.0},
 {'is_progress_report': True, 'confidence': 1.0},
 {'is_progress_report': True, 'confidence': 1.0},
 {'is_progress_report': True, 'confidence': 1.0},
 {'is_progress_report': True, 'confidence': 1.0},
 {'is_progress_report': True, 'confidence': 1.0},
 {'is_progress_report': True, 'confidence': 1.0},
 {'is_progress_report': True, 'confidence': 1.0},
 {'is_progress_report': True, 'confidence': 1.0},
 {'is_progress_report': True, 'confidence': 1.0},
 {'is_progress_report': True, 'confidence': 1.0},
 {'is_progress_report': True, 'confidence': 1.0},
 {'is_progress_report': True, 'confidence': 1.0},
 {'is_progress_report': True, 'confidence': 1.0},
 {'is_progress_report': True, 'confidence': 1.0},
 {'is_progress_report': True, 'confidence': 0.9},
 {'is_progress_report': True, 'confidence': 1.0},
 {'is_progress_report': True, 'confidence': 1.0},
 {'is_progress_report': True, 'confidence': 1.0},
 {'is_progress_report': True, 'confidence': 1.0},


In [15]:
extraction_outputs

[[{'book_text': 'Kej',
   'start_chapter': 1,
   'end_chapter': 2,
   'raw_text': 'Kej 1-2',
   'confidence': 1.0,
   'source': 'llm'}],
 [{'book_text': 'Kel',
   'start_chapter': 3,
   'end_chapter': 4,
   'raw_text': 'Kel 3-4 selesai',
   'confidence': 1.0,
   'source': 'llm'}],
 [{'book_text': 'Im',
   'start_chapter': 5,
   'end_chapter': 6,
   'raw_text': 'Im 5-6',
   'confidence': 1.0,
   'source': 'llm'}],
 [{'book_text': 'Bil',
   'start_chapter': 10,
   'end_chapter': 12,
   'raw_text': 'Bil 10-12',
   'confidence': 1.0,
   'source': 'llm'}],
 [{'book_text': 'Ul',
   'start_chapter': 6,
   'end_chapter': 8,
   'raw_text': 'Ul 6-8',
   'confidence': 1.0,
   'source': 'llm'}],
 [{'book_text': 'Yosua',
   'start_chapter': 1,
   'end_chapter': 4,
   'raw_text': 'Yos 1-4',
   'confidence': 1.0,
   'source': 'llm'}],
 [{'book_text': 'Hak',
   'start_chapter': 3,
   'end_chapter': 4,
   'raw_text': 'Hak 3-4',
   'confidence': 1.0,
   'source': 'llm'}],
 [{'book_text': 'Rut',
   'star